In [7]:
import pandas as pd
import geopandas as gpd
import requests
from zipfile import ZipFile

In [31]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')

%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編/"
n = int(start_pref)
while n <= int(finish_pref):
    #損害額のデータフレームをダウンロード
    damage_path = f'./dev/損害額/{n}_step8_1_output.csv'
    df_damage = pd.read_csv(damage_path)
    
    #貯蓄のデータフレームをダウンロード
    saving_path = f'./dev/貯蓄/{n}_saving_output.csv'
    df_saving = pd.read_csv(saving_path, encoding='shift-jis')
    
    #丁町字ポリゴンのデータフレームをダウンロード
    # if n < 10:
    #     with ZipFile(f'./data/神奈川県　行政区域/A002005212020DDSWC0{n}-JGD2011.zip', 'r') as zip:
    #         gdf_shp = zip.open('r2ka0{n}.shp', 'r')
    #         gdf_polygon = gpd.read_file(gdf_shp)
    # else:
    #     with ZipFile(f'./data/神奈川県　行政区域/A002005212020DDSWC{n}-JGD2011.zip', 'r') as zip:
    #         gdf_shp = zip.open('r2ka{n}.shp', 'r')
    #         gdf_polygon = gpd.read_file(gdf_shp)
    
    
    
    #KEYCODEの値毎に損害額を集計
    grouped_df = df_damage.groupby('KEY_CODE', as_index=False)['damage_price'].mean()
    grouped_df.rename(columns={'damage_price':'KEY_CODE_damage_price'}, inplace=True)

    
    
    #KEYCODEをキーに損害額と貯蓄のデータを空間結合
    merge_df =  pd.merge(grouped_df, df_saving[['KEY_CODE', 'saving']], on='KEY_CODE')
    #損害額を貯蓄額で除することで平均負担額を算出
    merge_df['ratio_of_burden'] = merge_df['KEY_CODE_damage_price'] / merge_df['saving']
    
    
    
    #ポリゴンデータのKEY_CODEがstr型のため変換
    # merge_df['KEY_CODE'] = merge_df['KEY_CODE'].astype(str)
    #ポリゴンデータを空間結合
    # merge_df = pd.merge(merge_df, gdf_polygon[['KEY_CODE', 'geometry']], on='KEY_CODE')
    # #ポリゴンデータにKEY_CODEの被りが多くあったので重複するものを削除
    # merge_df = merge_df.drop_duplicates(subset=['KEY_CODE'], keep='first')
    
    # #損害額のdfにmerge_dfを結合
    # df_building_damage = pd.merge(df_damage, merge_df, on='KEY_CODE')
    
    
    # merge_path = f'./dev/負担度/{n}_ratio_of_burden.csv' 
    # merge_df.to_csv(merge_path, index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コー : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [26]:
merge_df

,KEY_CODE,KEY_CODE_damage_price,saving,ratio_of_burden
0,331010030,9536814.425594777,1.253896e+07,0.760575
1,331010040,5795345.888650694,1.366290e+07,0.424167
2,331010050,2952936.727341513,1.076630e+07,0.274276
3,331010060,7935723.126438144,1.317969e+07,0.602118
4,331010070,4065107.146302262,1.481522e+07,0.274387
...,...,...,...,...
1715,33216010001,8950730.630324055,1.260938e+07,0.709847
1716,33216017001,12110510.361321187,1.229536e+07,0.984966
1717,33216017002,7242370.214399333,1.133333e+07,0.639033
1718,33216017003,11584331.89472784,1.219892e+07,0.949619


In [32]:
df_building_damage.head(200)

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price,KEY_CODE_damage_price,saving,ratio_of_burden
0,1573,133.797213,34.672247,0,166.805,1364,166.805,331012270,1,4,12,256000,0.036,25.5,0.6,4.270208e+07,4.452973e+06,1.074290e+07,1.212602e+07,0.885938
1,1574,133.797034,34.671509,0,131.640,1364,131.640,331012270,1,2,12,256000,0.036,8.0,0.6,3.369984e+07,1.497890e+07,1.074290e+07,1.212602e+07,0.885938
2,1695,133.797357,34.672400,0,96.400,1364,96.400,331012270,1,3,12,256000,0.036,15.5,0.6,2.467840e+07,7.370945e+06,1.074290e+07,1.212602e+07,0.885938
3,1688,133.797788,34.673460,0,184.710,1364,184.710,331012270,1,3,11,256000,0.036,15.5,0.4,4.728576e+07,9.415541e+06,1.074290e+07,1.212602e+07,0.885938
4,1694,133.797064,34.672907,0,102.940,1364,102.940,331012270,1,1,11,256000,0.036,5.0,0.4,2.635264e+07,8.833405e+06,1.074290e+07,1.212602e+07,0.885938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,4619,133.806740,34.671892,0,116.290,1364,116.290,331012280,1,3,11,256000,0.036,15.5,0.4,2.977024e+07,5.927850e+06,9.386909e+06,1.348742e+07,0.695975
196,4620,133.806771,34.672095,0,97.050,1364,97.050,331012280,1,1,11,256000,0.036,5.0,0.4,2.484480e+07,8.327977e+06,9.386909e+06,1.348742e+07,0.695975
197,4676,133.804631,34.672489,0,170.385,1364,170.385,331012280,1,6,11,256000,0.036,45.5,0.4,4.361856e+07,8.723712e+05,9.386909e+06,1.348742e+07,0.695975
198,4710,133.805594,34.673482,0,140.430,1364,140.430,331012280,1,1,11,256000,0.036,5.0,0.4,3.595008e+07,1.205047e+07,9.386909e+06,1.348742e+07,0.695975


In [21]:
gdf_polygon[gdf_polygon['KEY_CODE'] == '142070410']

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,N_KEN,N_CITY,KIGO_I,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry
4039,142070410,14,207,041000,神奈川県,茅ヶ崎市,萩園,NaN,8101,2106450.102,...,NaN,NaN,NaN,102,9610,3939,139.379664,35.345095,0410-00,"POLYGON ((139.39137 35.35103, 139.39056 35.350..."
